# Creating HF Dataset for Mistral Fine-tuning

Code authored by: Shaw Talebi <br>
Video link: https://youtu.be/XpoKB3usmKc <br>
Blog link: https://medium.com/towards-data-science/qlora-how-to-fine-tune-an-llm-on-a-single-gpu-4e44d6b5be32 <br>
<br>
Colab link: https://colab.research.google.com/drive/1AErkPgDderPW0dgE230OOjEysd0QV1sR?usp=sharing <br>
Dataset link: https://huggingface.co/datasets/shawhin/shawgpt-youtube-comments <br>
Model link: https://huggingface.co/shawhin/shawgpt-ft

In [1]:
import csv
import random
from datasets import Dataset, DatasetDict

/Users/hussen/Desktop/Year 4/Semester 2/IL181/Assignments/Assignment 2/nlp/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/hussen/Desktop/Year 4/Semester 2/IL181/Assignments/Assignment 2/nlp/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### prep training examples

In [25]:
# load csv of YouTube comments
comment_list = []
response_list = []

with open('data/reddit-comments.csv', mode ='r') as file:
    file = csv.reader(file)
    
    # read file line by line
    for line in file:
        # skip first line
        if line[0]=='Comment':
            continue
            
        # append comments and responses to respective lists
        comment_list.append(line[0])
        response_list.append(line[1])

In [3]:
# attempt 1 at prompt format
# intstructions_string = f"""ShawGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. \
# It reacts to feedback aptly and ends responses with its signature '–ShawGPT'. \
# ShawGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, \
# thus keeping the interaction natural and engaging.
# """

# example_template = lambda comment, response: f'''<s>[INST] {intstructions_string} \nViewer: {comment} \nShawGPT: [/INST]''' + response + "</s>"

# example_list = []
# for i in range(len(comment_list)):
#     example = example_template(comment_list[i],response_list[i])
#     example_list.append(example)

# print(example_list[10])

In [26]:
# attempt 2 at prompt format
instructions_string = f"""ShawGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. \
It reacts to feedback aptly and ends responses with its signature '–ShawGPT'. \
ShawGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, \
thus keeping the interaction natural and engaging.

Please respond to the following comment.
"""

example_template = lambda comment, response: f'''<s>[INST] {comment} \n[/INST]\n''' + response + "</s>"

example_list = []
for i in range(len(comment_list)):
    example = example_template(comment_list[i],response_list[i])
    example_list.append(example)

print(example_list[10])

<s>[INST] I was going to downvote you for refusing the reasonable request, but since you were impolite back, you get my upvote instead.   \n  \n faggot 
[/INST]
that is better</s>


In [27]:
print(example_list[0])

<s>[INST] Yup. Insane payments. Not sustainable. But the banks are all in 
[/INST]
But what if they only give loans to unemployed people? That way, the banks wouldn't have any risk, since 6 x 0 is 0.</s>


In [28]:
instructions_string = f"""FunGPT is an engaging and witty AI assistant trained on Reddit comment data. Its responses are designed 
to be entertaining, humorous, and unpredictable, while still being relevant to the user's comments. FunGPT communicates in a casual, 
conversational tone, often employing sarcasm, wordplay, and pop culture references. It aims to keep interactions light-hearted and fun, 
occasionally pushing boundaries with edgy or risqué humor (but never crossing ethical lines).

Please respond to the following comment in an engaging and entertaining way.
"""

example_template = lambda comment, response: f'''<s>[INST] \n{comment} \n[/INST]\n''' + response + "</s>"

example_list = []
for i in range(len(comment_list)):
    example = example_template(comment_list[i], response_list[i])
    example_list.append(example)

print(example_list[10])

<s>[INST] 
I was going to downvote you for refusing the reasonable request, but since you were impolite back, you get my upvote instead.   \n  \n faggot 
[/INST]
that is better</s>


In [29]:
# create train/test split
test_index_list = random.sample(range(0, len(example_list)-1), 9)

test_list = [example_list[index] for index in test_index_list]

for example in test_list:
    example_list.remove(example)

### create HF dataest

In [30]:
data = DatasetDict({'train':Dataset.from_dict({"example":example_list}), 'test':Dataset.from_dict({"example":test_list})})

In [31]:
data

DatasetDict({
    train: Dataset({
        features: ['example'],
        num_rows: 240
    })
    test: Dataset({
        features: ['example'],
        num_rows: 9
    })
})

In [32]:
dataset_name = 'reddit_comments'
data.push_to_hub(dataset_name)

Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  1.88it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/hussenmi/reddit_comments/commit/16b0f165df840b5c589c0f11b894263e173c1718', commit_message='Upload dataset', commit_description='', oid='16b0f165df840b5c589c0f11b894263e173c1718', pr_url=None, pr_revision=None, pr_num=None)

### push dataset to hub

In [24]:
# option 1: notebook login
from huggingface_hub import notebook_login
notebook_login()

# # option 2: key login
# from huggingface_hub import login
# write_key = 'hf_' # paste token here
# login(write_key)

In [10]:
# push dataset to hub
data.push_to_hub("shawhin/shawgpt-youtube-comments")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/531 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/shawhin/shawgpt-youtube-comments/commit/eb6e890103c25bb7f4be2d8ce541dd2b320d46f9', commit_message='Upload dataset', commit_description='', oid='eb6e890103c25bb7f4be2d8ce541dd2b320d46f9', pr_url=None, pr_revision=None, pr_num=None)